# CO Matchup Matrix
Matrix of CO win rates for each possible matchup. Each value represents the rate which the CO on the row wins against the CO on the column. Only includes matches with a player above 1400 std or fog score. Time outs and draws are excluded. COs are sorted by pick frequency, as a result, most matchups in the bottom right corner are missing data. 

In [7]:
import pandas as pd
import plotly.graph_objects as go
from utils import *

co_df = pd.read_csv("data/co_data.csv")
map_df = pd.read_csv("data/map_data.csv")
user_df = pd.read_csv("data/user_data.csv", index_col= 'userName')
replay_df = pd.read_csv("data/replay_data.csv", index_col= 'replayId')
pick_df = pd.read_csv("data/pick_data.csv", index_col= 'replayId')
ban_df = pd.read_csv("data/ban_data.csv", index_col= 'replayId')

In [8]:
# Filter high ranked players, >= 1400 represents 25%, top quartile
highranked_users = user_df[(user_df['stdScore'] >= 1400) | (user_df['fogScore'] >= 1400)]

# Replays played by high ranked players, exclude time out and draws
highranked_replays = replay_df.loc[pick_df[pick_df['userName'].isin(highranked_users.index)].index.unique()]
highranked_replays = highranked_replays[(highranked_replays['isBoot'] != True) & 
                                        (highranked_replays['winnerPlayerIndex'] != 0)]

In [9]:
# Matchup table for each CO
co_list = co_df['coName'].values
matchup_table_dict = {}

for co_name in co_list:
    matchup_table_dict[co_name] = get_co_matchup_table(co_name, pick_df, highranked_replays.index, highranked_replays)

In [10]:
# Determine CO frequency, used for CO display order
co_pick_frequency = pick_df.loc[highranked_replays.index, :]['coName'].value_counts()
co_pick_frequency = co_pick_frequency.drop('(No CO)')

In [11]:
all_co_matchup_table = pd.concat(matchup_table_dict.values(), keys= matchup_table_dict.keys())

# Removing (No CO)
all_co_matchup_table = all_co_matchup_table.drop(index= '(No CO)', level= 0)
all_co_matchup_table = all_co_matchup_table.drop(index= '(No CO)', level= 1)

# Sorting CO alphabetically
# all_co_matchup_table = all_co_matchup_table.sort_index()

# Sorting CO by pick frequency
all_co_matchup_table = all_co_matchup_table.reindex(co_pick_frequency.index, level= 0)
all_co_matchup_table = all_co_matchup_table.reindex(co_pick_frequency.index, level= 1)

In [12]:
co_matchup_matrix = all_co_matchup_table.unstack(1)

In [22]:
fig = go.Figure(
    go.Heatmap(
    z= co_matchup_matrix['matchupWinRate'].to_numpy(),
    x= co_matchup_matrix['matchupWinRate'].columns,
    y= co_matchup_matrix['matchupWinRate'].index,
    customdata= co_matchup_matrix['matchupTotal'].to_numpy(),
    colorscale= 'emrld',
    texttemplate= "%{z:.2f}",
    hovertemplate= (
        "matchupWinRate: %{z:.2f}<br>"
        "matchupTotal: %{customdata}"
        "<extra></extra>")
    )
)

fig.update_xaxes(
    title= "Losing CO",
    side= 'top',
)
fig.update_yaxes(
    title= "Winning CO",
    autorange= 'reversed',
)
fig.update_layout(
    title= ("<b>CO Matchup Win Rate Matrix</b>"),
    height= 1000,
    width= 1000,
    margin=dict(l=125, r=50, t=125, b=50),
)

fig.show()

In [23]:
# Save figure to html file
# fig.write_html("reports/co_matchup_matrix.html")